In [67]:
import geopandas as gpd
from owslib.wfs import WebFeatureService
from requests import Request
import subprocess
base_url = "https://geoserver.plandata.dk/geoserver/wfs" 
data_folder = './data/'
import json
import os



Call the WFS server al folect all layers as WFS

In [ ]:

wfs = WebFeatureService(url=base_url, version='2.0.0')

headers = {
    # ... your headers if any ...
}
for layer in wfs.contents:
    
    startIndex = 0
    count = 1000
    all_content = ''
    layer_file_name =layer.replace(':', '_')
    print(layer)
    while True:
        print("run")
        params = {
            "service": "WFS",
            "version": "2.0.0",
            "request": "GetFeature",
            "typeNames": layer,  # replace with the correct typeName
            'startIndex': startIndex,
            'count': count
        }
        
        response = requests.get(base_url, headers=headers, params=params)
        
        content = response.content.decode('utf-8')
        
        # If it's the first iteration, store everything up to the first <wfs:member> tag
        if startIndex == 0:
            all_content += content.split('<wfs:member>')[0]

        # Extract and store all <wfs:member> sections
        members = content.split('<wfs:member>')[1:]
        for member in members:
            all_content += '<wfs:member>' + member.split('</wfs:member>')[0] + '</wfs:member>'
        
        # Check if there's more to fetch
        if content.count('<wfs:member>') < count:
            break

        startIndex += count

    # Add the closing tag
    all_content += "</wfs:FeatureCollection>"

    # Save to a GML file
    with open(data_folder + layer_file_name +'.gml', 'w') as file:
        file.write(all_content)


lpd:lpd_bp_Stationer
run
lpd:lpd_bp_arealreservation
run
lpd:lpd_bp_byggezoner_stationer
run
lpd:lpd_bp_kompressorstation_everdrup
run
lpd:lpd_bp_modtageterminal_nybro
run
lpd:lpd_bp_oplag_og_arbplads
run
lpd:lpd_gaslf_arealreservation
run
lpd:lpd_gaslf_midlertidige_arbejdspladser
run
lpd:lpd_gaslf_stationer
run
lpd:lpd_gaslf_stationsareal
run
lpd:lpd_hoevsoere_maaleomraade_region
run
lpd:lpd_oesterild_maaleomraade_region
run
lpd:lpd_omplacering_sommerhusomr_vedt_2019_skal_tilbagefoeres
run
lpd:lpd_omplacering_sommerhusomr_vedt_2019_tilbagefoerelse
run
lpd:lpd_omplacering_sommerhusomr_vedt_2019_udlaeg
run
lpd:lpd_omplacering_sommerhusomr_vedt_2019_udlaeg_betinget
run
lpd:lpd_sommerhus_tilbagefoerelse_20190315
run
lpd:lpd_sommerhus_udlaeg_20190315
run
lpd:lpd_udviklingsomr_vedt_2019
run
lpd:lpd_udviklingsomr_vedt_2019_paragraf3
run
lpd:lpd_udviklingsomr_vedt_2019_paragraf5
run
lpd:lpd_udviklingsomr_vedt_2019_traadt_i_kraft
run
lpd:lpd_udviklingsomr_vedt_2021
run
lpd:lpd_udviklingsomr_ve

Procedures for vonverting the files

In [78]:

def generate_default_lookup(input_folder):
    lookup = {}
    for file in os.listdir(input_folder):
        if file.endswith('.gml'):
            layer_name = os.path.splitext(file)[0]
            # You can customize these default parameters as needed
            lookup[file] = {
                's_srs': 'EPSG:25832',
                't_srs': 'EPSG:25832',
                'nln': layer_name
            }
    return lookup


def convert_gml_to_geopackage(input_folder, output_gpkg, lookup_table):
    my_env = os.environ.copy()
    my_env["GML_SKIP_CORRUPTED_FEATURE"] = "YES"

    for gml_file in os.listdir(input_folder):
        print(gml_file)
        if gml_file.endswith('.gml'):
            gml_path = os.path.join(input_folder, gml_file)

            # Get parameters from lookup table or set to defaults
            params = lookup_table.get(gml_file, {
                's_srs': 'EPSG:25832',
                't_srs': 'EPSG:25832',
                'nln': 'merged_layer'
            })

            cmd = [
                'ogr2ogr',
                '-f', 'GPKG',
                '-update', '-append','-skipfailures',
                output_gpkg,
                gml_path,
                '-s_srs', params['s_srs'],
                '-t_srs', params['t_srs'],
                '-nln', params['nln']
            ]
            subprocess.run(cmd, env=my_env)

def save_lookup_to_file(lookup, filename="lookup_table.json"):
    with open(filename, 'w') as file:
        json.dump(lookup, file, indent=4)

def load_lookup_from_file(filename="lookup_table.json"):
    if os.path.exists(filename):
        with open(filename, 'r') as file:
            return json.load(file)
    return None


Generate and save a lookup table for file conversion

In [72]:
lookup = generate_default_lookup(data_folder)
save_lookup_to_file(lookup,filename='plandata.json')

Load the lookuo table and run the conversion

In [79]:
lookup = load_lookup_from_file(filename="plandata.json")
convert_gml_to_geopackage(data_folder, 'output.gpkg', lookup)

pdk_theme_pdk_kommuneplantillaeg_forslag_vvm_v.gml
pdk_theme_pdk_bevaringsvaerdigelandskaber_vedtaget_v.gml
lpd_theme-lpd-vaerloese-flyvestation.gml
pdk_theme_pdk_transformationsomraade_forslag_v.gml


ERROR 1: XML parsing of GML file failed : not well-formed (invalid token) at line 2, column 1


pdk_theme_pdk_kommuneplantillaeg_aflyst_v.gml
pdk_theme_pdk_codelist_status_v.gml
pdk_theme_pdk_lokalplan_forslag_vindmoelle_v.gml
pdk_theme_pdk_mulighedforudtraedenkloakopland_vedtaget_v.gml
lpd_theme-lpd-sommerhusomraade-til-byzone.gml
ekstern_tbb_hems.gml
pdk_theme_pdk_anvendelsevandsoekyst_forslag_v.gml
lpd_lpd_udviklingsomr_vedt_2021_paragraf3.gml
pdk_theme_pdk_kommuneplantillaeg_helekommunen_forslag_v.gml
pdk_theme_pdk_udbygningsaftale_aflyst_v.gml
pdk_theme_pdk_byggefelt_vedtaget_version_v.gml


FAILURE:
Unable to open datasource `./data/pdk_theme_pdk_udbygningsaftale_aflyst_v.gml' with the following drivers.
  -> `FITS'
  -> `PCIDSK'
  -> `netCDF'
  -> `PDS4'
  -> `VICAR'
  -> `JP2OpenJPEG'
  -> `PDF'
  -> `MBTiles'
  -> `BAG'
  -> `EEDA'
  -> `OGCAPI'
  -> `ESRI Shapefile'
  -> `MapInfo File'
  -> `UK .NTF'
  -> `LVBAG'
  -> `OGR_SDTS'
  -> `S57'
  -> `DGN'
  -> `OGR_VRT'
  -> `Memory'
  -> `CSV'
  -> `NAS'
  -> `GML'
  -> `GPX'
  -> `LIBKML'
  -> `KML'
  -> `GeoJSON'
  -> `GeoJSONSeq'
  -> `ESRIJSON'
  -> `TopoJSON'
  -> `Interlis 1'
  -> `Interlis 2'
  -> `OGR_GMT'
  -> `GPKG'
  -> `SQLite'
  -> `WAsP'
  -> `PostgreSQL'
  -> `OpenFileGDB'
  -> `DXF'
  -> `CAD'
  -> `FlatGeobuf'
  -> `Geoconcept'
  -> `GeoRSS'
  -> `VFK'
  -> `PGDUMP'
  -> `OSM'
  -> `GPSBabel'
  -> `OGR_PDS'
  -> `WFS'
  -> `OAPIF'
  -> `EDIGEO'
  -> `SVG'
  -> `Idrisi'
  -> `XLS'
  -> `ODS'
  -> `XLSX'
  -> `Elasticsearch'
  -> `Carto'
  -> `AmigoCloud'
  -> `SXF'
  -> `Selafin'
  -> `JML'
  -> `PLSCENES'

lpd_theme-lpd-aflastningsomraader-hovedstadsomraadet.gml
pdk_theme_pdk_oversvoemerosion_aflyst_v.gml
pdk_theme_pdk_codelist_oekologiskforbindelse_v.gml
lpd_lpd_bp_modtageterminal_nybro.gml
pdk_theme_pdk_lavbundsareal_vedtaget_v.gml
pdk_theme_pdk_tilslutningspligtomraade_vedtaget_v.gml
